<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания № 16


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс PaymentMethod в C#, который будет представлять 
различные способы оплаты. На основе этого класса разработать 2-3 производных 
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из 
классов должны быть реализованы новые атрибуты и методы, а также 
переопределены некоторые методы базового класса для демонстрации 
полиморфизма.
##### Требования к базовому классу PaymentMethod:
• Атрибуты: ID способа оплаты (PaymentMethodId), Название способа оплаты 
(MethodName), Минимальная сумма (MinAmount).

 • Методы:

o ProcessPayment(decimal amount): метод для обработки платежа 
указанной суммы.

o CheckMinimumAmount(decimal amount): метод для проверки 
минимальной суммы платежа.

o GetPaymentDetails(): метод для получения деталей способа оплаты.
##### Требования к производным классам:
1. ОнлайнОплата (OnlinePayment): Должен содержать дополнительные 
атрибуты, такие как URL платежной системы (PaymentUrl). 
Метод ProcessPayment() должен быть переопределен для включения URL 
платежной системы в процесс оплаты.
2. БанковскийПеревод (BankTransfer): Должен содержать дополнительные 
атрибуты, такие как Банковские данные (BankData). 
Метод CheckMinimumAmount() должен быть переопределен для проверки 
минимальной суммы платежа с учетом банковских комиссий.
3. Наличные (CashPayment) (если требуется третий класс): Должен содержать 
дополнительные атрибуты, такие как Место выдачи наличных 
(CashPickupPoint). Метод GetPaymentDetails() должен быть переопределен 
для отображения места выдачи наличных.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте простое, сложное и множественное наследование

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [4]:
public interface IPaymentMethod 
{
    void ProcessPayment(decimal amount);
    bool CheckMinimumAmount(decimal amount);
    string GetPaymentDetails();
    decimal CalculateCommission(decimal amount);  
}

public interface ICashback
{
    decimal CalculateCashback(decimal amount); 
}

public interface IRefund
{
    void ProcessRefund(decimal amount);
}

public interface IAddNum
{
    string AddNum(string str);
}

public abstract class PaymentMethod : IPaymentMethod
{
    public int PaymentMethodId { get; set; }
    public string MethodName { get; set; }
    public decimal MinAmount { get; set; }
    
    public virtual decimal AddNum(decimal amount)
    {
        return amount += 1;
    }

    public PaymentMethod(int paymentMethodId, string methodName, decimal minAmount)
    {
        PaymentMethodId = paymentMethodId;
        MethodName = methodName;
        MinAmount = minAmount;

    }

    public abstract void ProcessPayment(decimal amount);

    public virtual bool CheckMinimumAmount(decimal amount)
    {
        return amount >= MinAmount;
    }

    public virtual string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount}";
    }

     public abstract decimal CalculateCommission(decimal amount);

}

public class Added : PaymentMethod, IAddNum
{
    public Added(int paymentMethodId, string methodName, decimal minAmount, int amount, string str): base(paymentMethodId, methodName, minAmount)
    {  
    }

    public override decimal AddNum(decimal amount)
    {
        return amount;
    }

    public string AddNum(string str)
    {
        return str;
    }

    public override void ProcessPayment(decimal amount)
    {
    }

    public override decimal CalculateCommission(decimal amount)
    {
        return 0;
    }

}

public class OnlinePayment : PaymentMethod
{
    public string PaymentUrl { get; set; }
    public string ConfirmationCode { get; set; } 

    public OnlinePayment(int paymentMethodId, string methodName, decimal minAmount, string paymentUrl)
        : base(paymentMethodId, methodName, minAmount)
    {
        PaymentUrl = paymentUrl;
    }

    public override void ProcessPayment(decimal amount)
    {
        decimal commission = CalculateCommission(amount);
        Console.WriteLine($"Онлайн-платеж {amount} через {PaymentUrl}, Код подтверждения: {ConfirmationCode}");
    }

    public override decimal CalculateCommission(decimal amount)
    {
        return amount * 0.02m;  
    }

    public string GenerateConfirmationCode()
    {
        ConfirmationCode = new Random().Next(1000, 9999).ToString();
        Console.WriteLine($"Сгенерирован код подтверждения: {ConfirmationCode}");
        return ConfirmationCode;
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, URL: {PaymentUrl}";
    }
}


public class BankTransfer : PaymentMethod
{
    public string BankData { get; set; }

    public BankTransfer(int paymentMethodId, string methodName, decimal minAmount, string bankData)
        : base(paymentMethodId, methodName, minAmount)
    {
        BankData = bankData;
    }

    public override void ProcessPayment(decimal amount)
    {
        decimal commission = CalculateCommission(amount);
        Console.WriteLine($"Банковский перевод на сумму {amount} через {BankData}. Комиссия: {commission} руб.");
    }

    public override decimal CalculateCommission(decimal amount)
    {
        return 50m;  
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, через: {BankData}, фиксированная комиссия: 50 руб.";
    }
}


public class CashPayment : PaymentMethod, ICashback, IRefund
{
    public string CashPickupPoint { get; set; }
    public decimal CashbackAccrual { get; set; }
    private decimal _paidAmount = 0;

    public CashPayment(int paymentMethodId, string methodName, decimal minAmount, string cashPickupPoint, decimal cashbackAccrual)
        : base(paymentMethodId, methodName, minAmount)
    {
        CashPickupPoint = cashPickupPoint;
        CashbackAccrual = cashbackAccrual;
    }

    public override void ProcessPayment(decimal amount)
    {
        _paidAmount += amount;
        decimal cashback = CalculateCashback(amount);
        Console.WriteLine($"Оплата наличными {amount}. Место выдачи: {CashPickupPoint}. Кэшбек: {cashback}");
    }

    public override decimal CalculateCommission(decimal amount)
    {
        return 0; 
    }

    public decimal CalculateCashback(decimal amount)
    {
        return amount * CashbackAccrual / 100;
    }

    public void ProcessRefund(decimal amount)
    {
        if (amount > _paidAmount)
        {
            Console.WriteLine("Ошибка: сумма возврата превышает сумму платежа.");
        }
        else
        {
            _paidAmount -= amount; 
            Console.WriteLine($"Возврат на сумму {amount} выполнен. Остаток: {_paidAmount}.");
        }
    }

    public override string GetPaymentDetails()
    {
        return $"{base.GetPaymentDetails()}, Место выдачи наличных: {CashPickupPoint}, Кэшбек: {CashbackAccrual}%";
    }
}


OnlinePayment onlinePayment = new OnlinePayment(1, "Онлайн-оплата", 100, "https://payment.example.com");
BankTransfer bankTransfer = new BankTransfer(2, "Банковский перевод", 100, "Сбербанк");
CashPayment cashPayment = new CashPayment(3, "Оплата наличными", 50, "Магазин", 2);

decimal amountToPay = 120;

if (onlinePayment.CheckMinimumAmount(amountToPay))
{
    onlinePayment.GenerateConfirmationCode();
    onlinePayment.ProcessPayment(amountToPay);
}
else
{
    Console.WriteLine($"Минимальная сумма для {onlinePayment.MethodName} не достигнута");
}

if (bankTransfer.CheckMinimumAmount(amountToPay))
{
    bankTransfer.ProcessPayment(amountToPay);
}
else
{
    Console.WriteLine($"Минимальная сумма для {bankTransfer.MethodName} не достигнута");
}
decimal amountBack = 50;
if (cashPayment.CheckMinimumAmount(amountToPay))
{
    cashPayment.ProcessPayment(amountToPay);
    Console.WriteLine($"Кэшбек за оплату: {cashPayment.CalculateCashback(amountToPay)}");
    cashPayment.ProcessRefund(amountBack);
}
else
{
    Console.WriteLine($"Минимальная сумма для {cashPayment.MethodName} не достигнута");
}

Console.WriteLine(onlinePayment.GetPaymentDetails());
Console.WriteLine(bankTransfer.GetPaymentDetails());
Console.WriteLine(cashPayment.GetPaymentDetails());
